In [23]:
import pandas as pd
import geocoder
import folium 
from geopy.geocoders import Nominatim
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
import re
import urllib

In [24]:
df = pd.read_csv('harare_Neighbourhoods.csv')

In [25]:
neighbourhoodlongitude =[]
neighbourhoodlatitude = []
for index, row in df.iterrows():
    url = '{0}{1}{2}'.format('http://nominatim.openstreetmap.org/search.php?q=',
                             row['Neighbourhoods']+', Harare',
                             '&format=json&polygon=0')
    response = requests.get(url).json()
    if(len(response) == 0):
        indexNames = df[ df['Neighbourhoods'] == row['Neighbourhoods']].index
        df.drop(indexNames,inplace=True)
    else:
        response = response[0]
        lst = [response.get(key) for key in ['lat','lon']]
        output = [float(i) for i in lst]
        neighbourhoodlatitude.append(output[0])
        neighbourhoodlongitude.append(output[1])
df['Latitude'] = neighbourhoodlatitude
df['Longitude'] = neighbourhoodlongitude
df.head()

,Neighbourhoods,Latitude,Longitude
0,Emerald Hill,-17.765920,31.026607
1,Glen Norah,-17.905015,30.968854
2,Glen View,-17.909441,30.943362
3,Greendale,-17.812800,31.127918
4,Hatfield,-17.878416,31.085835


In [26]:
#Coordinates of Harare
address = 'Harare'

geolocator = Nominatim(user_agent="dc_explorer")
hre_location = geolocator.geocode(address)
hre_latitude = hre_location.latitude
hre_longitude = hre_location.longitude
hre_latitude, hre_longitude

(-17.831773, 31.045686)

In [27]:
map_hre = folium.Map(location=[hre_latitude, hre_longitude], zoom_start=12)
# add markers to map
for lat, lng, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Neighbourhoods']):
    label = neighbourhood
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='navy',
        fill=True,
        fill_color='royalblue',
        fill_opacity=0.7,
        parse_html=False).add_to(map_hre)  
    
map_hre

In [28]:
#foursquare
CLIENT_ID = 'VOXWGHQXQ5DCSNPU3YX35YQ2XJEJKH0TXYLSPVK3SFFGUB4Z' 
CLIENT_SECRET = 'FRZEOJ0H14ALOWB2FRPJDOZS22WAUYF3EXH3BAGTWY4HRHP0' 
VERSION = '20180605'
LIMIT = 100 

In [29]:
def getNearbyVenues(neighbourhood, latitudes, longitudes, categoryIds, radius):
    try:
        hre_venues_list=[]
        for name, lat, lng in zip(neighbourhood, latitudes, longitudes):
            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT, categoryIds)
            # make the GET request
            response = requests.get(url).json()
            results = response['response']['venues']
            # return only relevant information for each nearby venue
            for venues in results:
                success = False
                try:
                    category = venues['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    hre_venues_list.append([(name, lat, lng, venues['name'], venues['location']['lat'], venues['location']['lng'],
                        venues['categories'][0]['name']
                    )])

        hre_nearby_venues = pd.DataFrame([item for hre_venue_list in hre_venues_list for item in hre_venue_list])
        hre_nearby_venues.columns = ['Neighbourhoods', 'Latitude', 'Longitude', 'Venue_Name', 'Venue_Latitude', 'Venue_Longitude', 'Venue_Category']
    except:
        print(url)

    return(hre_nearby_venues)

In [30]:
hre_restaurants_venues = getNearbyVenues(neighbourhood=df['Neighbourhoods'], latitudes=df['Latitude'],
                                         longitudes=df['Longitude'], categoryIds='4d4b7105d754a06374d81259', radius=1000)
hre_restaurants_venues

,Neighbourhoods,Latitude,Longitude,Venue_Name,Venue_Latitude,Venue_Longitude,Venue_Category
0,Emerald Hill,-17.765920,31.026607,The Rose Cafe,-17.765243,31.032058,Café
1,Emerald Hill,-17.765920,31.026607,chicken inn Emerald hill,-17.764449,31.034974,Fried Chicken Joint
2,Emerald Hill,-17.765920,31.026607,Pizza Inn - Emerald Hill,-17.769303,31.033895,Pizza Place
3,Emerald Hill,-17.765920,31.026607,Creamy Inn Emerald hill,-17.769235,31.033879,Dessert Shop
4,Emerald Hill,-17.765920,31.026607,Total Bonjour Shop,-17.769272,31.033830,Food Court
...,...,...,...,...,...,...,...
273,Northwood,-17.754263,31.044634,Chicken Basket,-17.761135,31.051240,Fast Food Restaurant
274,Northwood,-17.754263,31.044634,Wild Tree Cafe,-17.761297,31.047611,Café
275,Vainona,-17.755365,31.077795,Greek Sizzler,-17.760351,31.081226,Greek Restaurant
276,Vainona,-17.755365,31.077795,Bel cantos,-17.759346,31.081787,Restaurant


In [31]:
#plot Restaurants
map_hre_restaurants = folium.Map(location=[hre_latitude, hre_longitude], zoom_start=12)

# add markers to map
for venue_latitude, venue_longitude, neighbourhood, venue_name in zip(hre_restaurants_venues['Venue_Latitude'], hre_restaurants_venues['Venue_Longitude'], hre_restaurants_venues['Neighbourhoods'], hre_restaurants_venues['Venue_Name']):
    label = '{} - {}'.format(venue_name, neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [venue_latitude, venue_longitude],
        radius=5,
        popup=label,
        color='maroon',
        fill=True,
        fill_color='lightcoral',
        fill_opacity=0.7,
        parse_html=False).add_to(map_hre_restaurants)  
    
map_hre_restaurants

In [32]:
#nearby highschools
venues_schools = getNearbyVenues(neighbourhood=df['Neighbourhoods'], latitudes=df['Latitude'], 
                                     longitudes=df['Longitude'], categoryIds='4bf58dd8d48988d13b941735',radius=500)
venues_schools

,Neighbourhoods,Latitude,Longitude,Venue_Name,Venue_Latitude,Venue_Longitude,Venue_Category
0,Emerald Hill,-17.765920,31.026607,Gateway Primary School,-17.768054,31.028821,College Quad
1,Mabelreign,-17.782319,31.000768,Alfred Beit Primary School,-17.785490,31.000867,Middle School
2,Chisipite,-17.788451,31.118514,Chisipite Senior School,-17.788161,31.122833,High School
3,Rolf Valley,-17.766172,31.111855,St John's Prep,-17.763453,31.111519,School
4,Meyrick Park,-17.811334,31.032365,Blakiston Primary School,-17.809282,31.037394,Elementary School
5,Park Town,-34.057685,18.673492,Molo Mhlaba,-34.059024,18.671276,School
6,Park Town,-34.057685,18.673492,Khayelitsha Football Centre For Hope,-34.054898,18.670215,School
7,Northwood,-17.754263,31.044634,North Park Primary School (Mt Pleasent),-17.756570,31.039430,Elementary School
8,Vainona,-17.755365,31.077795,Vainona Primary School,-17.757548,31.077149,Elementary School


In [33]:
#plot high schools
map_hre_schools = folium.Map(location=[hre_latitude, hre_longitude], zoom_start=12)

# add markers to map
for venue_latitude, venue_longitude, neighbourhood, venue_name in zip(venues_schools['Venue_Latitude'], venues_schools['Venue_Longitude'], venues_schools['Neighbourhoods'], venues_schools['Venue_Name']):
    label = '{} - {}'.format(venue_name, neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [venue_latitude, venue_longitude],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='honeydew',
        fill_opacity=0.7,
        parse_html=False).add_to(map_hre_schools)  
    
map_hre_schools

In [15]:
#nearby universities 
venues_university = getNearbyVenues(neighbourhood=df['Neighbourhoods'], latitudes=df['Latitude'],
                                    longitudes=df['Longitude'], categoryIds='4bf58dd8d48988d1ae941735', radius=1000)
venues_university

,Neighbourhoods,Latitude,Longitude,Venue_Name,Venue_Latitude,Venue_Longitude,Venue_Category
0,Mount Pleasant,-17.773289,31.052805,University of Zimbabwe,-17.780273,31.051177,University
1,Mount Pleasant,-17.773289,31.052805,University of Zimbabwe,-17.783262,31.054831,University
2,Mount Pleasant,-17.773289,31.052805,UZ - Graduate Business School,-17.780173,31.051296,University
3,Alexandra Park,-17.831773,31.045686,Africa UNIVERSITY Harare Campus,-17.831514,31.051163,University
4,Sunridge,-17.831773,31.045686,Africa UNIVERSITY Harare Campus,-17.831514,31.051163,University


In [16]:
#plot universities
map_hre_university = folium.Map(location=[hre_latitude, hre_longitude], zoom_start=12)

# add markers to map
for venue_latitude, venue_longitude, neighbourhood, venue_name in zip(venues_university['Venue_Latitude'], venues_university['Venue_Longitude'], venues_university['Neighbourhoods'], venues_university['Venue_Name']):
    label = '{} - {}'.format(venue_name, neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [venue_latitude, venue_longitude],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='honeydew',
        fill_opacity=0.7,
        parse_html=False).add_to(map_hre_university)  
    
map_hre_university

In [34]:
#nearby offices 
venues_offices = getNearbyVenues(neighbourhood=df['Neighbourhoods'], latitudes=df['Latitude'],
                                    longitudes=df['Longitude'], categoryIds='4bf58dd8d48988d124941735', radius=500)
venues_offices.head()

,Neighbourhoods,Latitude,Longitude,Venue_Name,Venue_Latitude,Venue_Longitude,Venue_Category
0,Emerald Hill,-17.765920,31.026607,Old Mutual Park,-17.767650,31.024649,Coworking Space
1,Mount Pleasant,-17.773289,31.052805,Kingdom People Church - 111 The Chase,-17.772411,31.049534,Office
2,Milton Park,-17.816513,31.022473,The Jupiter Drawing Room (Harare),-17.814512,31.020886,Office
3,Milton Park,-17.816513,31.022473,Mawazo Brandevelopment,-17.814566,31.020919,Office
4,Highlands,-17.798104,31.090195,Inkosi Prints,-17.796734,31.090886,Office


In [18]:
#plot offices
map_hre_offices= folium.Map(location=[hre_latitude, hre_longitude], zoom_start=12)

# add markers to map
for venue_latitude, venue_longitude, neighbourhood, venue_name in zip(venues_offices['Venue_Latitude'], venues_offices['Venue_Longitude'], venues_offices['Neighbourhoods'], venues_offices['Venue_Name']):
    label = '{} - {}'.format(venue_name, neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [venue_latitude, venue_longitude],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='honeydew',
        fill_opacity=0.7,
        parse_html=False).add_to(map_hre_offices)  
    
map_hre_offices

In [19]:
#merging datasets
def mergedf(hre_venues, df, newcolname):
    grouped_hre_venues = pd.DataFrame()
    grouped_hre_venues=df.groupby('Neighbourhoods').count()
    for neighbourhoods in hre_venues['Neighbourhoods']:
        if((grouped_hre_venues.index).contains(neighbourhoods)):
            hre_venues.loc[hre_venues['Neighbourhoods'] == neighbourhoods,newcolname] = grouped_hre_venues.loc[neighbourhoods, 'Venue_Name']
    return hre_venues

In [35]:
hre_venues = pd.DataFrame()
hre_venues = df.copy()
hre_venues = mergedf(hre_venues, hre_restaurants_venues, 'Restaurant')
hre_venues = mergedf(hre_venues, venues_schools, 'Schools')
hre_venues = mergedf(hre_venues, venues_university, 'Universities')
hre_venues = mergedf(hre_venues, venues_offices, 'Offices')
hre_venues.fillna(value=0,inplace=True)
hre_venues.head()

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: The 'contains' method is deprecated and will be removed in a future version. Use 'key in index' instead of 'index.contains(key)'
  


,Neighbourhoods,Latitude,Longitude,Restaurant,Schools,Universities,Offices
0,Emerald Hill,-17.765920,31.026607,7.0,1.0,0.0,1.0
1,Glen Norah,-17.905015,30.968854,0.0,0.0,0.0,0.0
2,Glen View,-17.909441,30.943362,0.0,0.0,0.0,0.0
3,Greendale,-17.812800,31.127918,5.0,0.0,0.0,0.0
4,Hatfield,-17.878416,31.085835,1.0,0.0,0.0,0.0


In [21]:
#Assigning weights to categories
weight_restaurant = -1 # negative weight to avoid concurrance
weight_schools = 1 #high school customers
weight_universities = 1.5 # university customers
weight_offices = 2 #work customers

In [22]:
weighted_venues = hre_venues[['Neighbourhoods']].copy()
weighted_venues['Score'] = hre_venues['Restaurant'] * weight_restaurant +hre_venues['Schools'] * weight_schools + hre_venues['Universities'] * weight_universities + hre_venues['Offices'] * weight_offices
weighted_venues = weighted_venues.sort_values(by=['Score'], ascending=False)
weighted_venues

,Neighbourhoods,Score
10,Mount Pleasant,6.5
19,Highlands,6.0
44,Dandaro Village,5.0
24,Graniteside,4.0
45,Park Town,3.0
18,Milton Park,3.0
25,Southerton,2.0
42,Adylinn,2.0
49,Amby,2.0
0,Emerald Hill,1.0
